<a href="https://colab.research.google.com/github/nahubn1/Hybrid-Robot-Navigation-System/blob/main/notebooks/results_analysis/Comparative_Inference_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup
The following cells mount Google Drive, clone the project repository and install requirements.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
COLAB_PROJECT_ROOT = '/content/Thesis_Project'
if not os.path.exists(COLAB_PROJECT_ROOT):
  !git clone https://github.com/nahubn1/Hybrid-Robot-Navigation-System {COLAB_PROJECT_ROOT}
os.chdir(COLAB_PROJECT_ROOT) # Change directory into the project
!git pull # Ensure it's the latest version

Cloning into '/content/Thesis_Project'...
remote: Enumerating objects: 1080, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 1080 (delta 82), reused 36 (delta 36), pack-reused 912 (from 1)
Receiving objects: 100% (1080/1080), 499.90 KiB | 2.08 MiB/s, done.
Resolving deltas: 100% (670/670), done.
Already up to date.


In [3]:
!pip install -r environment/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [4]:
import sys
# Add src to path
sys.path.append(os.path.join(os.getcwd(), 'src'))

# Define Drive paths
DRIVE_DATA_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/data'
DRIVE_MODELS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/models'
DRIVE_RESULTS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/results'

# Link Drive storage to local cloned directories
if not os.path.islink('data'):
    !ln -s {DRIVE_DATA_PATH} data
if not os.path.islink('models'):
    !ln -s {DRIVE_MODELS_PATH} models
if not os.path.islink('results'):
    !ln -s {DRIVE_RESULTS_PATH} results

print("✅ Environment setup complete.")

✅ Environment setup complete.


# Comparative Inference Analysis

In [5]:

from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader

from dnn_guidance.data_loader import PathfindingDataset
from dnn_guidance.inference import InferenceHandler
from dnn_guidance.model import UNetFiLM, HRFiLMNet, ResNetFPNFiLM
from utils.visualization import plot_inference_comparison


In [6]:
# Map model names to (class, checkpoint path)
MODEL_REGISTRY = {
    'UNet-FiLM': (UNetFiLM, DRIVE_RESULTS_PATH / Path('checkpoints/unet_film_v2_dice_heavy/unet_film_v2_dice_heavy_best_model.pth')),
    'HR-FiLM-Net': (HRFiLMNet, DRIVE_RESULTS_PATH / Path('checkpoints/hr_film_net_v1_baseline/hr_film_net_v1_baseline_best_model.pth')),
    'ResNet-FPN-FiLM': (ResNetFPNFiLM, DRIVE_RESULTS_PATH / Path('checkpoints/resnet_fpn_film_v2/resnet_fpn_film_v2_best_model.pth')),
}

In [7]:
# Create test DataLoader
samples_dir = DRIVE_DATA_PATH / Path('training_data')
gt_dir = DRIVE_DATA_PATH / Path('ground_truth')
dataset = PathfindingDataset(samples_dir, gt_dir, augment=False)
loader = DataLoader(dataset, batch_size=1, shuffle=False)


In [12]:
num_samples_to_visualize = 5
random_indices = np.random.choice(len(dataset), num_samples_to_visualize, replace=False)

for i in random_indices:
    (grid_tensor, robot_tensor), _ = dataset[i] # Get sample by index
    grid = grid_tensor.squeeze(0).numpy()[2]
    robot = robot_tensor.squeeze(0).numpy()
    predictions_to_plot = {}
    for name, (cls, ckpt) in MODEL_REGISTRY.items():
        handler = InferenceHandler(cls, ckpt, device='cpu')
        heatmap = handler.predict(grid, robot)
        predictions_to_plot[name] = heatmap
    plot_inference_comparison({
        'input_grid': grid,
        'ground_truth_heatmap': np.zeros_like(grid),
        'predictions': predictions_to_plot,
    })

torch.Size([4, 200, 200])
torch.Size([4, 200, 200])
torch.Size([4, 200, 200])
torch.Size([4, 200, 200])
torch.Size([4, 200, 200])


Analysis notes go here.